In [ ]:
import psycopg2
import csv
import io
import sys
from urllib.request import Request
from urllib.request import urlopen

conn = psycopg2.connect(dbname="", user="", password="")
cur = conn.cursor()

cur.execute("""
select * from
(
select 
    jsonb_array_elements(metadata->'resources')->>'url' as url,
    jsonb_array_elements(metadata->'resources')->>'identifier' as identifier, 
    jsonb_array_elements(metadata->'resources')->'schema' as schema,
    jsonb_array_elements(metadata->'resources')->>'format' as resource_format
from package
) t where t.resource_format = 'CSV'
""")
            
data = cur.fetchall()

In [ ]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def create_column_str(columns_metadata):
    columns = []
    for column_metadata in columns_metadata:
        col_name = column_metadata['name']
        col_type = column_metadata['type']
        if col_type == 'numeric' or col_type == 'text':
            columns.append(col_name + ' ' + col_type)
        elif col_type == 'datetime':
            if column_metadata['format'] == 'YYYY-[H]H': # strange format, make it text
                columns.append(col_name + ' ' + 'text')
            else:
                columns.append(col_name + ' ' + 'timestamp')
    return '(' + ','.join(columns) + ')'

def create_table_name(id):
    return 'table_' + id.replace('-','_')

def insert_table(table_name, columns_metadata, reader, cur):   
    for j, row in enumerate(reader):
        if j > 0:
            col_values = []
            for i, column_metadata in enumerate(columns_metadata):
                col_type = column_metadata['type']
                col_data = row[i]
                if col_type == 'datetime':
                    col_format = column_metadata['format']
                    if col_format == 'YYYY':
                        col_data += '-01-01'
                    elif col_format == 'YYYY-MM':
                        col_data += "-01"
                    col_data = "'" + col_data + "'"
                elif col_type == 'text':
                    col_data = "'" + col_data + "'"
                elif col_type == 'numeric' and not is_number(col_data):
                    col_data = "0" #this is very bad, fix me!
                col_values.append(col_data)
            sql = 'insert into ' + table_name + ' values (' + ','.join(col_values) + ')'
            cur.execute(sql)
            
                          
def access_resource(resource_url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    req = Request(resource_url, headers=hdr)
    response = urlopen(req).read().decode('utf-8')
    return csv.reader(io.StringIO(response))

success = 0
failed = 0
for row in data[:10]:
    url = row[0]
    id = row[1]
    columns_metadata = row[2]
    table_name = create_table_name(id)
    sql_str = "CREATE TABLE " + table_name + " " + create_column_str(columns_metadata)
    drop_str = "DROP TABLE IF EXISTS " + table_name
    print(sql_str)
    cur.execute(drop_str)
    cur.execute(sql_str)
    reader = access_resource(url)
    try:
        insert_table(table_name, columns_metadata, reader, cur)
        success += 1
    except:
        print("Unexpected error:", sys.exc_info()[0])
        failed += 1
        
print('done!')

conn.commit()
cur.close()
conn.close()

In [196]:
conn.commit()
cur.close()
conn.close()

In [155]:
from datetime import datetime
datetime_object = datetime.strptime('2014-02','%Y-%m')

In [156]:
datetime_object

datetime.datetime(2014, 2, 1, 0, 0)

In [197]:
success

20

In [198]:
failed

0

In [2]:
data

[('https://storage.data.gov.sg/3g-public-cellular-mobile-telephone-services/resources/call-success-rate-2013-2016-2016-08-29T09-22-31Z.csv',
  'c9f714db-cc3b-49ad-bb0f-302ba33505ee',
  [{'format': 'YYYY-MM',
    'name': 'month',
    'sub_type': 'month',
    'title': 'Month',
    'type': 'datetime'},
   {'name': 'telco', 'sub_type': 'general', 'title': 'Telco', 'type': 'text'},
   {'name': 'success_rate',
    'sub_type': 'general',
    'title': 'Success Rate',
    'type': 'numeric',
    'unit_of_measure': 'Percentage'}],
  'CSV'),
 ('https://storage.data.gov.sg/3g-public-cellular-mobile-telephone-services/resources/call-success-rate-2009-2011-2016-02-16T03-04-41Z.csv',
  'e9ec02c5-f3fe-49ec-960c-f7c05acf536c',
  [{'format': 'YYYY',
    'name': 'year',
    'sub_type': 'year',
    'title': 'Year',
    'type': 'datetime'},
   {'name': 'operators',
    'sub_type': 'general',
    'title': 'Operators',
    'type': 'text'},
   {'description': 'Percentages are expressed as a value over 100, i.e